# Autoencoder
* https://medium.com/ai-academy-taiwan/what-are-autoencoders-175b474d74d1
* https://morvanzhou.github.io/tutorials/machine-learning/keras/2-6-autoencoder/
* https://github.com/MorvanZhou/tutorials/blob/master/kerasTUT/9-Autoencoder_example.py

In [1]:
SAVE_NAME = '200_100dim'
HISTORY_SAVE = './history/' + SAVE_NAME + '.csv'
WEIGHTS_SAVE = './weight/' + SAVE_NAME + '.h5'

In [2]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [4]:
from keras.models import Model
from keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Load data

In [5]:
df = pd.read_csv('./input/followingMatrix_over1.csv')
df

,movie,20thcenturyfox,a24,accessonline,adamsandler,adamshankman,agt,aldis_hodge,alecbaldwininsta,alexandrashipppp,...,willpowerpacker,willsmith,wmag,woodyharrelson,wwe,yahooentertainment,zacefron,zaziebeetz,zendaya,zoeydeutch
0,21bridgesmovie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47metersdown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,abeautifuldaymovie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,abominablemovie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,adastramovie,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,wrinklestheclown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162,xmenmovies,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
163,yardiefilm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,yesterdaymovie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x_li = []
y_li = []
for i, row in df.iterrows():
    y_li.append(row['movie'])
    x_li.append(row.iloc[1:].tolist())
    
x = np.asarray(x_li)
y = np.asarray(y_li)
print(x.shape, y.shape)

(166, 491) (166,)


In [7]:
from sklearn.preprocessing import MinMaxScaler
print(x)
scaler = MinMaxScaler()
scaler.fit(x)
train_scaled = scaler.transform(x)
print(train_scaled.shape)
print(train_scaled)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
(166, 491)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(132, 491) (34, 491)
(132,) (34,)


In [9]:
input_vec = 491
encoding_dim = 50

In [10]:
input_dim = Input(shape = (input_vec, ))

# Encoder Layers
encoded1 = Dense(100, activation = 'relu')(input_dim)
encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)

# Decoder Layers
decoded1 = Dense(100, activation = 'relu')(encoded2)
decoded2 = Dense(input_vec, activation = 'sigmoid')(decoded1)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded2)
autoencoder.compile(optimizer = 'Adam', loss = 'binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 491)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               49200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_4 (Dense)              (None, 491)               49591     
Total params: 108,941
Trainable params: 108,941
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss')]

history = autoencoder.fit(X_train, X_train, nb_epoch = 100, batch_size = 50, validation_data = (X_test, X_test),shuffle = True, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.


/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 132 samples, validate on 34 samples
Epoch 1/100


InternalError: Blas GEMM launch failed : a.shape=(30, 491), b.shape=(491, 100), m=30, n=100, k=491
	 [[{{node dense_1/MatMul}}]]
	 [[{{node Mean}}]]

In [ ]:
autoencoder.save(WEIGHTS_SAVE)

In [ ]:
# save history
# acc = history.history['acc']
# val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

his_df = pd.DataFrame(data = {
    'Epoch' : epochs,
    'Loss' : loss,
#     'Acc' : acc,
    'Val_loss' : val_loss#,
#     'Val_acc' : val_acc
})
his_df = his_df[['Epoch', 'Loss','Val_loss']]
his_df.to_csv(HISTORY_SAVE, index = False)

In [ ]:
encoder = Model(inputs = input_dim, outputs = encoded2)

encoded_train = pd.DataFrame(encoder.predict(train_scaled))
userContent=np.array(encoded_train)

encoded_train = encoded_train.add_prefix('feature_')
print(encoded_train.shape)
encoded_train #.head()

In [ ]:
encoded_train['movie'] = df['movie']
encoded_train = encoded_train.set_index('movie')
encoded_train

In [ ]:
encoded_train.to_csv('./output/' + SAVE_NAME + '.csv')

In [ ]:
from sklearn import cluster, datasets, metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

silhouette_avgs = []
ks = range(2, 30)

for k in ks:
    kmeans_fit = cluster.KMeans(n_clusters = k).fit(encoded_train)
    cluster_labels = kmeans_fit.labels_
    silhouette_avg = metrics.silhouette_score(train_scaled, cluster_labels)
    silhouette_avgs.append(silhouette_avg)

# 作圖並印出 k = 2 到 30 的績效
plt.bar(ks, silhouette_avgs)
plt.show()
print(silhouette_avgs)